# Descriptive Statistics

In [8]:
def read_csv(filepath):
    '''
    Read the events.csv and mortality_events.csv files. 
    Variables returned from this function are passed as input to the metric functions.
    '''
    events = pd.read_csv(filepath + 'events.csv')
    mortality = pd.read_csv(filepath + 'mortality_events.csv')
    return events, mortality

def event_count_metrics(events, mortality):
    '''
    Implement this function to return the event count metrics.
    Event count is defined as the number of events recorded for a given patient.
    '''
    events_2 = events.copy()
    events_2 = events_2.assign(deceased=events_2['patient_id'].isin(mortality['patient_id']).astype(int))
    surviveData = events_2.copy()
    surviveData = surviveData.loc[surviveData['deceased'] == 0]
    deceasedData = events_2.copy()
    deceasedData = deceasedData.loc[deceasedData['deceased'] == 1]

    deceasedEventCount = deceasedData['event_id'].count()
    deceasedDistincPatient = deceasedData['patient_id'].nunique()
    deceasedData['Agg'] = deceasedData.groupby('patient_id')['patient_id'].transform('count')
    
    Meandeceased_EventC = deceasedEventCount/deceasedDistincPatient
    Maxdeceased_EventC = deceasedData['Agg'].max()
    Mindeceased_EventC = deceasedData['Agg'].min()

    surviveEventCount = surviveData['event_id'].count()
    surviveDistincPatient = surviveData['patient_id'].nunique()
    surviveData['Agg'] = surviveData.groupby('patient_id')['patient_id'].transform('count')

    Meansurvive_EventC = surviveEventCount/surviveDistincPatient
    Maxsurvive_EventC = surviveData['Agg'].max()
    Minsurvive_EventC = surviveData['Agg'].min()

    return Mindeceased_EventC, Maxdeceased_EventC, Meandeceased_EventC, Minsurvive_EventC, Maxsurvive_EventC, Meansurvive_EventC

def encounter_count_metrics(events, mortality):
    '''
    Implement this function to return the encounter count metrics.
    Encounter count is defined as the count of unique dates on which a given patient visited the ICU. 
    '''
    events_2 = events.copy()
    events_2 = events_2.assign(deceased=events_2['patient_id'].isin(mortality['patient_id']).astype(int))
    surviveData = events_2.copy()
    surviveData = surviveData.loc[surviveData['deceased'] == 0]
    surviveData = surviveData.drop(['deceased', 'event_id', 'event_description', 'value'], axis=1)
    surviveData = surviveData.drop_duplicates()
    deceasedData = events_2.copy()
    deceasedData = deceasedData.loc[deceasedData['deceased'] == 1]
    deceasedData = deceasedData.drop(['deceased', 'event_id', 'event_description', 'value'], axis=1)
    deceasedData = deceasedData.drop_duplicates()
    
    deceased_timestamp = deceasedData['timestamp'].count()
    deceased_patients = deceasedData['patient_id'].nunique()
    deceasedData['Agg'] = deceasedData.groupby('patient_id')['patient_id'].transform('count')
    
    Meandeceased_EncounC = deceased_timestamp/deceased_patients
    Maxdeceased_EncounC = deceasedData['Agg'].max()
    Mindeceased_EncounC = deceasedData['Agg'].min()

    survive_timestamp = surviveData['timestamp'].count()
    survive_patients = surviveData['patient_id'].nunique()
    surviveData['Agg'] = surviveData.groupby('patient_id')['patient_id'].transform('count')
    
    Meansurvive_EncounC = survive_timestamp/survive_patients
    Maxsurvive_EncounC = surviveData['Agg'].max()
    Minsurvive_EncounC = surviveData['Agg'].min()

    return Mindeceased_EncounC, Maxdeceased_EncounC, Meandeceased_EncounC, Minsurvive_EncounC, Maxsurvive_EncounC, Meansurvive_EncounC

def record_length_metrics(events, mortality):
    '''
    Implement this function to return the record length metrics.
    Record length is the duration between the first event and the last event for a given patient. 
    '''
    events_2 = events.copy()
    events_2['timestamp'] = pd.to_datetime(events_2['timestamp'])    
    LengthMetrics = events_2[['patient_id','timestamp']].groupby('patient_id').agg([max,min])
    LengthMetrics['duration'] = LengthMetrics['timestamp']['max']-LengthMetrics['timestamp']['min']
    LengthMetrics['days']=LengthMetrics['duration'].dt.days
    LengthMetrics=LengthMetrics.reset_index()
    LengthMetrics = LengthMetrics[['patient_id', 'days']]
    deathData1 = LengthMetrics[LengthMetrics['patient_id'].isin(mortality['patient_id'])]    
    deathData = deathData1['days']  
    Meandeceased_LenMetrics = deathData.mean()
    Maxdeceased_LenMetrics = deathData.max()
    Mindeceased_LenMetrics = deathData.min()
    surviveData = LengthMetrics[-LengthMetrics['patient_id'].isin(mortality['patient_id'])]
    surviveData = surviveData['days']
    Meansurvive_LenMetrics = surviveData.mean()
    Maxsurvive_LenMetrics = surviveData.max()
    Minsurvive_LenMetrics = surviveData.min()    
    return Mindeceased_LenMetrics, Maxdeceased_LenMetrics, Meandeceased_LenMetrics, Minsurvive_LenMetrics, Maxsurvive_LenMetrics, Meansurvive_LenMetrics

def main():
    
    train_path = 'data/'
    events, mortality = read_csv(train_path)

    #event count metrics
    print("output for the event count metrics is :") 
    print(event_count_metrics(events, mortality))
    print()
    #encounter count metrics    
    print("output for the encounter count metrics is :")
    print(encounter_count_metrics(events, mortality)) 
    print()
    #record length metrics    
    print("output for the record length metrics is :")
    print( record_length_metrics(events, mortality))
    
    
if __name__ == "__main__":
    main()

output for the event count metrics is :
(1, 8635, 982.014, 1, 12627, 498.118)

output for the encounter count metrics is :
(1, 203, 23.038, 1, 391, 15.452)

output for the record length metrics is :
(0, 1972, 127.532, 0, 2914, 159.2)


# Data ETL Pipeline

In [9]:
#Feature construction
import utils
import pandas as pd
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler


def read_csv(filepath):    
    '''    
    Read the events.csv, mortality_events.csv and event_feature_map.csv files into events, mortality and feature_map.
    Return events, mortality and feature_map
    '''
    #Columns in events.csv - patient_id,event_id,event_description,timestamp,value
    events = pd.read_csv(filepath + 'events.csv')
    
    #Columns in mortality_event.csv - patient_id,timestamp,label
    mortality = pd.read_csv(filepath + 'mortality_events.csv')

    #Columns in event_feature_map.csv - idx,event_id
    feature_map = pd.read_csv(filepath + 'event_feature_map.csv')
    return events, mortality, feature_map



In [10]:
def calculate_index_date(events, mortality, deliverables_path):
    
    '''
    1. Create list of patients alive ( mortality_events.csv only contains information about patients deceased)
    2. Split events into two groups based on whether the patient is alive or deceased
    3. Calculate index date for each patient
    
    IMPORTANT:
    Save indx_date to a csv file in the deliverables folder named as etl_index_dates.csv. 
    Use the global variable deliverables_path while specifying the filepath. 
    Each row is of the form patient_id, indx_date.
    The csv file should have a header 
    For example if you are using Pandas, you could write: 
        indx_date.to_csv(deliverables_path + 'etl_index_dates.csv', columns=['patient_id', 'indx_date'], index=False)

    Return indx_date
    '''
    events_3 = events.copy()
    events_3 = events_3.assign(deceased=events_3['patient_id'].isin(mortality['patient_id']).astype(int))
    surviveData = events_3.copy()
    surviveData = surviveData.loc[surviveData['deceased'] == 0]
    surviveData = surviveData.drop(['deceased', 'event_id', 'event_description', 'value'], axis=1)
    surviveData = surviveData.drop_duplicates()
    surviveData['timestamp'] =pd.to_datetime(surviveData.timestamp)
    surviveDataFinal = surviveData.loc[surviveData.groupby('patient_id').timestamp.idxmax()]
    
    deceasedData = mortality.copy()
    deceasedData= deceasedData[['patient_id','timestamp']]
    deceasedData['timestamp']= pd.to_datetime(deceasedData['timestamp'])
    deceasedData['timestamp']= deceasedData['timestamp']-timedelta(days = 30)
    
    indx_date = pd.concat([surviveDataFinal,deceasedData]).reset_index(drop=True)
    indx_date = indx_date.rename(columns={'timestamp': 'indx_date'})
    indx_date.to_csv(deliverables_path + 'etl_index_dates.csv', columns=['patient_id', 'indx_date'], index=False)
    
    return indx_date

In [11]:
def filter_events(events, indx_date, deliverables_path):
    
    '''
    1. Join indx_date with events on patient_id
    2. Filter events occuring in the observation window(IndexDate-2000 to IndexDate)
    Save filtered_events to a csv file in the deliverables folder named as etl_filtered_events.csv. 
    Use the global variable deliverables_path while specifying the filepath. 
    Each row is of the form patient_id, event_id, value.
    The csv file should have a header 
    For example if you are using Pandas, you could write: 
        filtered_events.to_csv(deliverables_path + 'etl_filtered_events.csv', columns=['patient_id', 'event_id', 'value'], index=False)

    Return filtered_events
    '''

    
    filtered_events0 = pd.merge(events, indx_date, on = ['patient_id'])
    filtered_events0['indx_date'] = pd.to_datetime(filtered_events0['indx_date'])
    filtered_events0['timestamp'] = pd.to_datetime(filtered_events0['timestamp'])
    filtered_events = filtered_events0.copy()
    filtered_events = filtered_events[(filtered_events.timestamp <= filtered_events.indx_date) 
    & (filtered_events.timestamp >= filtered_events.indx_date-timedelta(days = 2000))]
    filtered_events.to_csv(deliverables_path + 'etl_filtered_events.csv', 
                           columns=['patient_id', 'event_id', 'value'], index=False)
    return filtered_events

In [12]:
def aggregate_events(filtered_events_df, mortality_df,feature_map_df, deliverables_path):
    
    '''
    steps:
    1. Replace event_id's with index available in event_feature_map.csv
    2. Remove events with n/a values
    3. Aggregate events using sum and count to calculate feature value
    4. Normalize the values obtained above using min-max normalization(the min value will be 0 in all scenarios) 
    
    IMPORTANT:
    Save aggregated_events to a csv file in the deliverables folder named as etl_aggregated_events.csv. 
    Use the global variable deliverables_path while specifying the filepath. 
    Each row is of the form patient_id, event_id, value.
    The csv file should have a header .
    For example if you are using Pandas, you could write: 
        aggregated_events.to_csv(deliverables_path + 'etl_aggregated_events.csv', columns=['patient_id', 'feature_id', 'feature_value'], index=False)

    Return aggregated_events
    '''
    aggregated_events0 = pd.merge(filtered_events_df, feature_map_df, on = 'event_id')    
    aggregated_events0 = aggregated_events0[['patient_id','idx','value']]
    aggregated_events0 = aggregated_events0[pd.notnull(aggregated_events0['value'])]  
    MedicineAgg = aggregated_events0[aggregated_events0['idx'] < 2680]    
    SumMedicineAgg = MedicineAgg.groupby(['patient_id','idx']).agg('sum').reset_index()
    SumMedicineAggMax = SumMedicineAgg.groupby(['idx']).agg('max').reset_index()
    SumMedicineAggMax = SumMedicineAggMax.rename(columns = {"value": "max"}) 
    SumMedicineAggMax = SumMedicineAggMax.drop(['patient_id'], axis=1)

    SumMedicineAgg = pd.merge(SumMedicineAgg, SumMedicineAggMax, on = 'idx')
    SumMedicineAgg["value2"] = SumMedicineAgg["value"]/SumMedicineAgg["max"]
    SumMedicineAgg = SumMedicineAgg.drop(['value', 'max'], axis=1)
    SumMedicineAgg = SumMedicineAgg.rename(columns = {"value2": "feature_value", "idx": "feature_id"})
    TestID = aggregated_events0[aggregated_events0['idx'] >= 2680]    
    TestAgg = TestID.groupby(['patient_id','idx']).agg('count').reset_index() 
    TestAggMax = TestAgg.groupby(['idx']).agg('max').reset_index()   
    TestAggMax = TestAggMax.rename(columns = {"value": "max"}) 
    TestAggMax = TestAggMax.drop(['patient_id'], axis=1)

    TestAgg = pd.merge(TestAgg, TestAggMax, on = 'idx')
    TestAgg["value2"] = TestAgg["value"]/TestAgg["max"]
    TestAgg = TestAgg.drop(['value', 'max'], axis=1)
    TestAgg = TestAgg.rename(columns = {"value2": "feature_value", "idx": "feature_id"})
    aggregated_events = pd.concat([SumMedicineAgg, TestAgg]).reset_index(drop = True)
    aggregated_events.to_csv(deliverables_path + 'etl_aggregated_events.csv', columns=['patient_id', 'feature_id', 'feature_value'], index=False)
    
    return aggregated_events

In [13]:
def create_features(events, mortality, feature_map):
    
    deliverables_path = 'output/'
    #Calculate index date
    indx_date = calculate_index_date(events, mortality, deliverables_path)
    #Filter events in the observation window
    filtered_events = filter_events(events, indx_date,  deliverables_path)    
    #Aggregate the event values for each patient 
    aggregated_events = aggregate_events(filtered_events, mortality, feature_map, deliverables_path)
    '''
    Complete the code below by creating two dictionaries - 
    1. patient_features :  Key - patient_id and value is array of tuples(feature_id, feature_value)
    2. mortality : Key - patient_id and value is mortality label
    '''
    patient_features = aggregated_events.groupby('patient_id')[['feature_id','feature_value']].apply(lambda x: [tuple(x) for x in x.values]).to_dict()
    events_3 = events.copy()
    events_3 = events_3.assign(deceased=events_3['patient_id'].isin(mortality['patient_id']).astype(int))
    surviveData = events_3.copy()
    surviveData = surviveData.loc[surviveData['deceased'] == 0]
    deceasedData = events_3.copy()
    deceasedData = deceasedData.loc[deceasedData['deceased'] == 1]
    #join two talbes
    ConcatedTable = pd.concat([surviveData, deceasedData]).reset_index(drop = True)
    ConcatedTable = ConcatedTable.drop(['event_id', 'event_description', 'timestamp', 'value'], axis=1) 
    mortality = pd.Series(ConcatedTable.deceased.values, index = ConcatedTable.patient_id).to_dict()
    return patient_features, mortality

In [14]:
def save_svmlight(patient_features, mortality, op_file, op_deliverable):
    
    '''
    Create two files:
    1. op_file - which saves the features in svmlight format. (See instructions in Q3d for detailed explanation)
    2. op_deliverable - which saves the features in following format:
       patient_id1 label feature_id:feature_value feature_id:feature_value feature_id:feature_value ...
       patient_id2 label feature_id:feature_value feature_id:feature_value feature_id:feature_value ...  
    
    Note: Please make sure the features are ordered in ascending order, and patients are stored in ascending order as well.     
    '''
    deliverable1 = open(op_file, 'wb')
    deliverable2 = open(op_deliverable, 'wb')
    for patient_id in sorted(patient_features):
        feature_pairs_string = ""
        for feature in sorted(patient_features[patient_id]):
            feature_pairs_string += " " + str(int(feature[0])) + ":" + format(feature[1], '.6f')
        svm_light_str = str(mortality[patient_id]) + feature_pairs_string + " \n"
        deliverable1.write(bytes((svm_light_str),'UTF-8')); #Use 'UTF-8'
        deliverable2.write(bytes((str(int(patient_id)) + " " + svm_light_str),'UTF-8'));

In [15]:
def main():
    train_path = 'data/train/'
    events, mortality, feature_map = read_csv(train_path)
    patient_features, mortality = create_features(events, mortality, feature_map)
    save_svmlight(patient_features, mortality, 'output/features_svmlight.train', 'output/features.train')

if __name__ == "__main__":
    main()

## output for model building
features_svmlight.train,features.train,etl_index_dates.csv, etl_filtered_events.csv, etl_aggregated_events